In [1]:
import sys
import os
sys.path.append("../../TychePlot/")
pixelvalidation=False
if pixelvalidation:
    %matplotlib inline
else:
    import matplotlib as mpl
    mpl.use("pgf")
import matplotlib.pyplot as plt
from Plot_processing import calc, export_data, calc_plot_list, add_to_plot
from PDF import PDF
from OLEDPlot import OLEDPlot
from SpectraPlot import SpectraPlot
from Plot import Plot
from Filereader import fileToNpArray
import dill
import copy as copy

In [2]:
#Constants

## OLED
Voltage=1
Current=2
Current_density=3
Luminance=4
Radiance=5
Current_Efficacy=6
Luminous_Efficacy=7
EQE=8
POWER=9

fill="_"
jvl_file_format=OLEDPlot.jvl_file_format_default
spectra_file_format=OLEDPlot.spectral_data_format_default

## SPECTRA
wavelength=1
intensity_wavelength=2
spectralRadiance_wavelength=3
energy=4
intensity_energy=5
spectralRadiance_energy=6

In [3]:
#Parameters

## rte
multithreaded=True

## files OLED
series_indicator="VII"
series_name="PVKDopiongi"
series_sample_qty=8
jvl_subdir="jvl/"
jvl_concat_subdir="jvl/"

valid_pixel_file="valid_pixel.bin"
OLED_output_file_path="jvl_plot/"


## files SPECTRA
spectra_subdir="../20210720_LXXXIX_DDAF_KTFSi/spectra/" # formerly input_file_path
SPECTRA_output_file_path="spectra_plot/"
corr_output_file_path="corr/"
fileformat={"separator":"\t", "skiplines":53, "fileEnding":".txt", "codec":"iso-8859-1"}


## figure
figscale=0.49
present_width_pt=665.79451
present_HW_ratio=1#0.82

## plot data

# 50 µl CsPbBr 10mg/ml mixed with 25/50/100 ul 10mg/ml LiTFSi (CB:DMF;9:1) diluted with 425/400/300 µl Hexane for 2:1, 1:1, 1:2 solutions


## plot data
labelList=[
    
        "native",# CH8000 4000rpm 30s, 20nm PVK (3000rpm 3mg/ml), CsPbBr3 DDAF 3-2 1mg/ml 500rpm, 55nm B3PYMPM, 0.5nm LiF, Al
        "1wt LiTFSi", 
        "1wt doped PVK + native",
        "1wt doped PVK + 1wt LiTFSi",
        "2.5wt doped PVK + native",# CH8000 4000rpm 30s, 20nm PVK (3000rpm 3mg/ml), CsPbBr3 DDAF 3-2 1mg/ml 500rpm, 55nm B3PYMPM, 0.5nm LiF, Al
        "2.5wt doped PVK + 1wt LiTFSi", 
        "5wt doped PVK + native",
        "5wt doped PVK + 1wt LiTFSi",
]

pixelsizes=[
    ##
        3.8, #
        3.8, #
        3.8, 
        3.8, #
    ##
        3.8, #
        3.8, #
        3.8, 
        3.8, #
]

spectra=[
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    ("1-spectrum-c-spectral-irradiance",2),
    #("1-spectrum-g-spectral-irradiance",2),
]


eqe_thresh=0.1

In [4]:
OLED_present={
            #"customFontsize":[3,3,1.5,2,2],
            "fig_width_pt":present_width_pt*figscale,
            "scaleX":(present_width_pt*figscale)/OLEDPlot.fig_width_default_pt,
            "fixedFigWidth":True,
            "HWratio":present_HW_ratio,
            "titleBool":False,
            "showColLabel": ["","","","", "", "","","","", ""],
            #"legendBool":False,
            "filenamePrefix":OLED_output_file_path+"presentation/",
            "labelPad":2.5,
            "axRect":[0.175,0.175,0.65,0.65]
}

SPECTRA_present={
            #"customFontsize":[3,3,1.5,2,2],
            "fig_width_pt":present_width_pt*figscale,
            "scaleX":(present_width_pt*figscale)/SpectraPlot.fig_width_default_pt,
            "fixedFigWidth":True,
            "HWratio":present_HW_ratio,
            "titleBool":False,
            #"legendBool":False,
            "filenamePrefix":SPECTRA_output_file_path+"presentation/",
            "labelPad":2.5,
            "axRect":[0.175,0.175,0.65,0.65]
}

In [5]:
OLED_inputParameters={
                    "spectraFile":"",
                    "labels":[],
                    "titleBool":False,
                    "customFontsize":[12,12,8,10,10],
                    "averageMedian":True,
                    "noSweepBackMeasured":True,
                    "averageSweepBack":False,
                    "useTex":True,
                    "filenamePrefix":OLED_output_file_path,
                    "pixelsize_mm2":3.8,
                    "showMarkers":True,
                    "iterMarkers":True,
                    "showLines":False,
                    "iterBoth":True,
                    "saveProps":{"saveAs":".png", "dpi":600},
                    #"fileFormat":{"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True},
                    #"append_col_in_label":False,
                    "spectralDataFormat":fileformat,
                    "showErrorOnlyEvery":2,
                    "showColAxType":["lin","lin","log","log","log","log","lin","lin","log","lin"],
                    #"idealDevice":1,
                    #"maxEqe":0.1,
                    #"curIdeal":True,
                    #"show":[[True,Tr1e],[False,False],[True,True]],
                    #"spectralDataFormat":{"separator":",", "skiplines":82, "fileEnding":".csv"}
                }

SPECTRA_inputParameters={
                        "titleBool":False,
                        "customFontsize":[12,12,8,10,10],
                        "filenamePrefix":SPECTRA_output_file_path,
                        #"normalizeMode":"custom",
                        "showFWHM":True,
                        "_filter":{"type":"savgol", "p1":41, "p2":3},
                        "ticklabelformat":"sci",
                        "fitColors":['#000000','#d62728','#2ca02c','#9467bd','#8c564b','#e377c2','#7f7f7f','#ff7f0e','#bcbd22','#17becf','#f8e520'],
                        "fileFormat":fileformat,
                        "append_col_in_label":False,
        
}

In [6]:
OLED_optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [-1,8],#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    [0,3]#POWER
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [10**-7,10**3],#Current_density
                                    [10**-2,10**8],#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    [10**-2,100],#EQE
                                    [0,5]#POWER
                                  ]
}

HO_optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [-6,6],#Current_density
                                    None,#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    None,#EQE
                                    None#POWER
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#Voltage
                                    None,#Current
                                    [10**-8,10**2],#Current_density
                                    [10**-2,10**8],#Lumiance
                                    None,#Radiance
                                    None,#Current_efficacy
                                    None,#Lumious_efficacy
                                    [0,4],#EQE
                                    None#POWER
                                  ]
}

SPECTRA_optionalParameters={
                    "customLims":True, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    [350,650],#wavelength
                                    [400,650],#Normalised Intensity
                                    [350,650],#spectralRadiance
                                    None,#energy
                                    None,#Normalised Intensity
                                    None,#reduced Intensity
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#wavelength
                                    None,#Normalised Intensity
                                    None,#spectralRadiance
                                    None,#energy
                                    None,#Normalised Intensity
                                    None,#reduced Intensity
                                  ],
                    "scaled":False
}

In [7]:

## OLED
OLED_desiredPlots=[
                {
                    "xCol":Voltage,
                    "yCol":Current_density,
                    "yCol2":Luminance,
                    "custom":
                    {
                        "injCode":"ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))\n",
                    }
                },   
                {
                    "xCol":Current_density,
                    "yCol":EQE,
                    "custom":
                    {
                        "xLim":[eqe_thresh,10000],
                        "xAxisLim":[10**-3,10**3],
                        "limCol":Luminance,
                        "legLoc":1,
                        "injCode":"",
                        "showErrorOnlyEvery":1,
                    }
                },
    {
                    "xCol":Luminance,
                    "yCol":EQE,
                    "custom":
                    {
                        "xLim":[eqe_thresh,10**6],
                        "xAxisLim":[10**-2,10**3],
                        "limCol":Luminance,
                        "legLoc":1,
                        "injCode":"",
                        "showErrorOnlyEvery":1,
                    }
                },
                    {
                    "xCol":Voltage,
                    "yCol":POWER,
                        "custom":
                    {
                        "showLines":True,
                        "showErrorOnlyEvery":1,
                        "append_col_in_label":False,
                    }
                                            
                },
]


HO_desiredPlots=[
                {
                    "xCol":Voltage,
                    "yCol":Current_density,
                    #"yCol2":Luminance,
                    "custom":
                    {
                        "injCode":"ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))\n"#+
                        #"ax2.plot(3.4,3*10**0,marker='o',ms=4.5,mfc='None',mec=(0.,0.,0.,1), zorder=4)\n"+
                        #"ax2.annotate('', xy=(4.7,3*10**0), xytext=(3.45,3*10**0), arrowprops=dict(arrowstyle='->', connectionstyle='arc3', linewidth=0.3), ha='center', va='center', zorder=4)\n"+
                        #"ax.plot(0.6,3*10**-4,marker='o',ms=4.5,mfc='None',mec=(0.,0.,0.,1), zorder=4)\n"+
                        #"ax.annotate('', xy=(-0.55,3*10**-4), xytext=(0.55,3*10**-4), arrowprops=dict(arrowstyle='->', connectionstyle='arc3', linewidth=0.3), ha='center', va='center', zorder=4)\n"
                    }
                },   
                    {
                    "xCol":Voltage,
                    "yCol":POWER,
                },
]

SPECTRA_desiredPlots=[
                {
                    "xCol":wavelength,
                    "xCol2":energy,
                    "yCol":intensity_wavelength,
                    "custom":
                    {
                        "rainbowMode":True
                    }
                },
                {
                    "xCol":wavelength,
                    "xCol2":energy,
                    "yCol":intensity_wavelength,
                    "custom":
                    {
                        "rainbowMode":False
                    }
                },
                {
                    "xCol2":wavelength,
                    "xCol":energy,
                    "yCol":intensity_energy,                    
                    
                }
]
SPECTRA_desiredPlots_comp=[
                {
                    "xCol":wavelength,
                    "xCol2":energy,
                    "yCol":intensity_wavelength,
                    #"yCol":spectralRadiance_wavelength,
                    "custom":
                    {
                        "rainbowMode":False
                    }
                },
                {
                    "xCol2":wavelength,
                    "xCol":energy,
                    "yCol":intensity_energy,                    
                    
                }
]

In [8]:
## OLED
OLED_groups=[
        (0,2,4,6,),
        (1,3,5,7),
]
OLED_group_names=[
                "native",
                "LiTFSi"
]
OLED_group_specific_input=[
                        {
                            "desiredPlots":OLED_desiredPlots,
                            "inputParameters":OLED_inputParameters,
                            "optionalParameters":OLED_optionalParameters,
                        },
]*len(OLED_groups)


## SPECTRA
SPECTRA_groups=[
        (2,3),
]
SPECTRA_group_names=[
            "CsPbBr3", #AI4083 3000rpm 30, MoOx 10nm, 4CzIPN 4nm, PVK (5mg/ml 4000rpm), CKP11-HPA 1000rpm, 40nm TPBi, Ca, Al  
]
SPECTRA_group_specific_input=[
{
                            "desiredPlots":SPECTRA_desiredPlots_comp,
                        },]*len(SPECTRA_groups)


In [9]:
len(OLED_group_specific_input)

2

In [10]:
#Dynamic Parameters
name=series_indicator+fill+series_name

In [11]:
concatenate = False
if concatenate:
    filelist=OLEDPlot.generateFileList(series_indicator, subdir=jvl_concat_subdir)
    fileList_conc=[]
    for device in filelist:
        subList2=[]
        for n in range(1,series_sample_qty+1):
            sublist=[]
            for pixel in device:
                if int(pixel[-1])==n:
                    sublist.append(pixel)
            if sublist != []:
                subList2.append(sublist)
        fileList_conc+=subList2
    fileList_conc=[concat_tup for concat_tup in fileList_conc if len(concat_tup)>1]

In [12]:
#[fileList_conc[0][-1]]+fileList_conc[0][:-1]

In [13]:
if concatenate:
    fileList_conc=fileList_conc[2:]
    fileList_conc=[[file[-1]]+file[:-1] for file in fileList_conc]
    fileList_conc

In [14]:
if concatenate:
    for file_tup in fileList_conc:
        OLEDPlot.concentenate_files(file_tup, fileFormat=OLEDPlot.jvl_file_format_default, subdir=jvl_subdir, previous_subdir=jvl_concat_subdir)

In [15]:
#getting valid_pixels (file or user_input)
try:
    with open(jvl_subdir+valid_pixel_file, "rb") as f:
        valid_info=dill.load(f)
except FileNotFoundError:
    valid_info=OLEDPlot.get_valid_pixel_by_user(series_indicator, samples=series_sample_qty, subdir=jvl_subdir, update_by_existing=True, jvl_file_format={"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True})
    with open(jvl_subdir+valid_pixel_file, "wb") as f:
        dill.dump(valid_info, f)
valid_pixel=valid_info[1]
valid_device=valid_info[0]

In [16]:
#valid_pixel[1]=[False,True,False,True]

In [17]:
#Importing files
#spectra=sorted([spectra_subdir+file for file in os.listdir(spectra_subdir) if spectra_file_format["fileEnding"] == file[-4:]])
OLED_fileList=OLEDPlot.generateFileList(series_indicator, samples=series_sample_qty, subdir=jvl_subdir, truthTable=valid_pixel, update_by_existing=True)
#del OLED_fileList[1][0]
OLED_fileList=sorted(OLED_fileList, key=lambda x:x[0][-3:])
#OLED_fileList=sorted(OLED_fileList, key=lambda x:x[0][9:13])
#OLED_fileList=[OLED_fileList[1]]+[OLED_fileList[0]]+[OLED_fileList[2]]
#del OLED_fileList[3][1]
#del OLED_fileList[3][0]
OLED_fileList

[['jvl/VII_A_1', 'jvl/VII_A_2', 'jvl/VII_A_3', 'jvl/VII_A_4'],
 ['jvl/VII_B_2', 'jvl/VII_B_3'],
 ['jvl/VII_C_2', 'jvl/VII_C_3'],
 ['jvl/VII_D_1', 'jvl/VII_D_2', 'jvl/VII_D_3'],
 ['jvl/VII_E_1', 'jvl/VII_E_2', 'jvl/VII_E_3'],
 ['jvl/VII_F_1', 'jvl/VII_F_2', 'jvl/VII_F_3'],
 ['jvl/VII_G_1', 'jvl/VII_G_2', 'jvl/VII_G_3'],
 ['jvl/VII_H_1', 'jvl/VII_H_2', 'jvl/VII_H_3']]

In [18]:
## OLED

plot_prop_list=[]
feature="both"
for group,group_name,specific_input in zip(OLED_groups,OLED_group_names,OLED_group_specific_input):
    fileList=[]
    spectra_loc=[]
    labels_loc=[]
    pixelsizes_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(desiredPlots)
    for pos in group:
        if isinstance(pos, tuple):
            fileList.append(OLED_fileList[pos[0]])
            labels_loc.append(labelList[pos[1]])
            pixelsizes_loc.append(pixelsizes[pos[3]])
        else:
            fileList.append(OLED_fileList[pos])
            labels_loc.append(labelList[pos])
            pixelsizes_loc.append(pixelsizes[pos])
        try:
            if type(spectra[pos]) == tuple:
                spectra_loc.append((spectra_subdir+spectra[pos][0],spectra[pos][1]))
            else:
                spectra_loc.append(spectra_subdir+spectra[pos])
        except:
            spectra_loc=""
            inputParameters_loc["spectralDataFormat"]={"fileEnding":""}
    inputParameters_loc["labels"]=labels_loc
    inputParameters_loc["spectraFile"]=spectra_loc
    inputParameters_loc["pixelsize_mm2"]=pixelsizes_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList,
             "desiredPlots":desiredPlots_loc,
             "present_params":OLED_present,
             "inputParameters":inputParameters_loc,
             "plot_class":OLEDPlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )
OLED_results=calc_plot_list(plot_prop_list, multithreaded=multithreaded)

plot_prop_list=[]
for group,group_name,specific_input in zip(SPECTRA_groups,SPECTRA_group_names,SPECTRA_group_specific_input):
    fileList_loc=[]
    spectra_loc=[]
    labels_loc=[]
    validYCol_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(SPECTRA_inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(SPECTRA_optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(SPECTRA_desiredPlots)
    for pos in group:
        if isinstance(pos, tuple):
            fileList_loc.append([spectra_subdir+spectra[pos[0]][0]])
            labels_loc.append(labelList[pos[1]])
            validYCol_loc.append([pos[2]])
        else:
            fileList_loc.append([spectra_subdir+spectra[pos][0]])
            labels_loc.append(labelList[pos])
            validYCol_loc.append([spectra[pos][1]])
    inputParameters_loc["labels"]=labels_loc
    inputParameters_loc["validYTable"]=validYCol_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList_loc,
             "desiredPlots":desiredPlots_loc,
             "present_params":SPECTRA_present,
             "inputParameters":inputParameters_loc,
             "plot_class":SpectraPlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )

#SPECTRA_results = calc_plot_list(plot_prop_list, multithreaded=multithreaded)

../../TychePlot/Data.py:140: UserWarning: Invalid end limit
  warnings.warn("Invalid end limit")
../../TychePlot/Data.py:140: UserWarning: Invalid end limit
  warnings.warn("Invalid end limit")
../../TychePlot/Data.py:140: UserWarning: Invalid end limit
  warnings.warn("Invalid end limit")
../../TychePlot/Data.py:140: UserWarning: Invalid end limit
  warnings.warn("Invalid end limit")


In [19]:
plot=OLED_results[0][1][0]

In [20]:
max_eqes=[]
for data, label in zip(plot.expectData,labelList):
    try:
        valid_eqe_start=data.getFirstIndexWhereGreaterOrEq(4,eqe_thresh, check_seq=5)
        valid_eqes=data.getData()[:,7][valid_eqe_start:]
        max_eqe=max(valid_eqes)
        max_eqes.append(max_eqe)
        print(f"Device {label}: Max EQE: {max_eqe:.3f} %")
    except IndexError:
        max_eqes.append(0)
        print(f"Device {label}: No EQE obtained, too low luminace!")

Device native: Max EQE: 0.005 %
Device 1wt LiTFSi: Max EQE: 0.874 %
Device 1wt doped PVK + native: Max EQE: 0.171 %
Device 1wt doped PVK + 1wt LiTFSi: Max EQE: 1.001 %


In [21]:
                        """axAnnotations":{
                            "ax1":{
                                "circle_pos":(4,3*10**0),
                                "circle_kwargs":{
                                                    "marker":'o',
                                                    "ms":4.5,
                                                    "mfc":'None',
                                                    "mec":(0.,0.,0.,1),
                                                    "zorder":4,
                                },
                                "arr_kwargs":{
                                    "xy":(4.7,3*10**0), 
                                    "xytext":(3.675,3*10**0), 
                                    "arrowprops":dict(arrowstyle='->', connectionstyle='arc3', linewidth=0.6),
                                    "ha":'center',
                                    "va":'center', 
                                    "zorder":4
                                },
                            },
                            "ax2":{
                                "circle_pos":(4,3*10**-1),
                                "circle_kwargs":{
                                                    "marker":'o',
                                                    "ms":20,
                                                    "mfc":'None',
                                                    "mec":(0.,0.,0.,1),
                                                    "zorder":4,
                                },
                                "arr_kwargs":{
                                    "xy":(4,3*10**-1), 
                                    "xytext":(4.5,3*10**-1), 
                                    "arrowprops":dict(arrowstyle='->', connectionstyle='arc3', linewidth=0.6),
                                    "ha":'center',
                                    "va":'center', 
                                    "zorder":4
                                },
                            }
                        }"""

'axAnnotations":{\n    "ax1":{\n        "circle_pos":(4,3*10**0),\n        "circle_kwargs":{\n                            "marker":\'o\',\n                            "ms":4.5,\n                            "mfc":\'None\',\n                            "mec":(0.,0.,0.,1),\n                            "zorder":4,\n        },\n        "arr_kwargs":{\n            "xy":(4.7,3*10**0), \n            "xytext":(3.675,3*10**0), \n            "arrowprops":dict(arrowstyle=\'->\', connectionstyle=\'arc3\', linewidth=0.6),\n            "ha":\'center\',\n            "va":\'center\', \n            "zorder":4\n        },\n    },\n    "ax2":{\n        "circle_pos":(4,3*10**-1),\n        "circle_kwargs":{\n                            "marker":\'o\',\n                            "ms":20,\n                            "mfc":\'None\',\n                            "mec":(0.,0.,0.,1),\n                            "zorder":4,\n        },\n        "arr_kwargs":{\n            "xy":(4,3*10**-1), \n            "xyte